### TextRank Text Summarization

---

<b>Goal</b>: Explore TextRank algorithm for text summary and use it with keras tokenizer

In [1]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.

Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.

W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.

Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.

Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.

Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

### ***0. Load libraries***

In [2]:
# Text imports
import re
import string
from langdetect import detect

# NLTK imports
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.cluster.util import cosine_distance

# NetworkX for TextRang Algorithm
import networkx as nx

# Keras text ans sequence imports
from tensorflow.keras.preprocessing import text as kpt
from tensorflow.keras.preprocessing import sequence

# Misc libraries
import numpy as np
import operator
from random import choice
from typing import List
from functools import reduce
from IPython.core.display import display, HTML

In [69]:
vector = List[float] # custom list element

def cos_sim(v: vector, w: vector) -> float:
    '''
    Calculate cosine similarity between two vectors. Return 0 if both vectors
    contain zeroes.

    Arguments:
        v:  vector
        w:  vector
    
    Returns:
        cosine similarity
    '''
    if ((sum(v) == 0) or (sum(w) == 0)):
        return 0
    else:
        return 1 - cosine_distance(v, w)


In [6]:
def retrieve_stop_words():
    '''
    Load stop words based on detected language

    Argumenst:
        None

    Returns:
        set of stop words 
    '''
    from spacy.lang.en.stop_words import STOP_WORDS as STOP_WORDS_en
    from spacy.lang.pl.stop_words import STOP_WORDS as STOP_WORDS_pl
    lang = detect(text)

    if lang == 'en':
        STOP_WORDS = STOP_WORDS_en
    elif lang == 'pl':
        STOP_WORDS = STOP_WORDS_pl
        return STOP_WORDS

In [7]:
STOP_WORDS = retrieve_stop_words()

In [8]:
class Preprocessing(object):
    '''
    Text cleaning and sentence tokenization class.
    '''
    def __init__(self, text):
        self.text = text
        self.oryg = text

    def lower(self):
        self.text = self.text.lower() 
        return self.text
    
    def remove_punctuation(self):
        self.text = self.text.translate(self.text.maketrans('', '', string.punctuation.replace('.', '')))
        return self.text 
    
    def remove_stop_words(self):
        self.text = ' '.join([word for word in self.text.split() if word not in STOP_WORDS])
        return self.text
    
    def remove_digits(self):
        self.text = re.sub(r'[\d+]', '', self.text)
        return self.text
    
    def sentence_tokenize(self):
        self.text = sent_tokenize(self.text)
        self.text = [sent.replace('.','') for sent in self.text]
        return self.text
    
    def basic_pipeline(self):
        self.lower()
        self.remove_digits()
        self.remove_punctuation()
        self.remove_stop_words()
        self.sentence_tokenize()
        return self.text

    def __call__(self):
        return self.text

In [11]:
def prepare_text(text: str) -> tuple:
    '''
    Preprocess the text, split it into sentences and return modified
    results as well as the original text, both as lists (split based on
    sentences)

    Arguments:
        text: original text
    
    Return
        tuple containing preprocessed text (as list of sentences) and 
        origianl text (as list of sentences)

    '''
    # make a class object
    cleaned_text = Preprocessing(text) 
    
    # preprocess and tokenize text into sentences
    sentences = cleaned_text.basic_pipeline() 

    # tokenize original text into sentences
    raw_sentences = Preprocessing(text).sentence_tokenize() 
    return sentences, raw_sentences

In [51]:
sentences, raw_sentences = prepare_text(text)

In [52]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

In [53]:
all_words = [elem.split() for elem in sentences]
all_words_flattened = reduce(operator.concat, all_words)

In [55]:
def make_representationX(sentences, max_top_words = 100, max_sent_len = 20):
    tokenizer = kpt.Tokenizer(num_words=max_top_words)
    tokenizer.fit_on_texts(sentences)
    seq = tokenizer.texts_to_sequences(sentences)
    return sequence.pad_sequences(seq, maxlen = max_sent_len, padding='post', truncating='post')

In [56]:
sim_matrix = np.zeros((len(sentences), len(sentences)))
sentences_repr = make_representationX(sentences)

for n, sent1 in enumerate(sentences):
    for m, sent2 in enumerate(sentences):
        if sent1 != sent2:
            a = sentences_repr[n]
            b = sentences_repr[m]

            sim_matrix[n][m] = cos_sim(a, b)    

In [57]:
scores = nx.pagerank(nx.from_numpy_array(sim_matrix))
scores

{0: 0.032075294104480984,
 1: 0.032202031514449385,
 2: 0.028105162417610255,
 3: 0.033277709501180745,
 4: 0.029008622088911532,
 5: 0.02945440618715988,
 6: 0.03013764769135546,
 7: 0.03468195854693096,
 8: 0.02391501449577009,
 9: 0.029444415452317154,
 10: 0.030429323764997865,
 11: 0.03044354950538758,
 12: 0.036829492049345956,
 13: 0.028133737287327515,
 14: 0.03167248500623338,
 15: 0.038630300996899986,
 16: 0.02125694049997876,
 17: 0.03140751316354146,
 18: 0.00464396286764706,
 19: 0.03044354950538758,
 20: 0.02603031682443926,
 21: 0.03276020651568418,
 22: 0.03763831269162044,
 23: 0.03044354950538758,
 24: 0.03044354950538758,
 25: 0.03044354950538758,
 26: 0.037433148902665386,
 27: 0.03044354950538758,
 28: 0.030205842636297744,
 29: 0.02455691741383579,
 30: 0.03683428794987795,
 31: 0.03148614002408098,
 32: 0.00464396286764706,
 33: 0.030443549505387576}

In [58]:
summary = sorted(((scores[i], s) for i,s in enumerate(raw_sentences)), reverse = True)

In [64]:
RATIO = 0.3

In [65]:
summarized_sents = [sent[1] for sent in summary[:int(len(summary) * RATIO)]]
summarized_sents

['Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy',
 'Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach',
 'Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec',
 'Falk von Kronberg nakrywa ich i każe aresztować',
 'Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii',
 'W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny',
 'Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi',
 'Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice)',
 'Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice',
 '\nGłównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w

In [66]:
def display_highlights(raw_sentences, summarized_sents):
    final = []
    for sent in raw_sentences:
        if sent not in summarized_sents:
            final += sent
        else:
            final += f'<span style="background-color:rgba(255,215,0,0.3);"> {sent} </span>'
    display(HTML(''.join([elem for elem in final])))

In [67]:
display_highlights(raw_sentences, summarized_sents)